In [ ]:
from azure.storage.blob import BlobServiceClient
import json
import pandas as pd

# Connect to Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=sg092620240215;AccountKey=+PaTF6WCZ0NY63Hni1XIWRJfWsnTI7QJCLVP0f1OXUoVzJyl0AcE4h2Pe1b7ZbgldGkDDFA0j9iK+AStvU4auA==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
silver_client = blob_service_client.get_container_client("silver")

# Load and flatten data
all_rosters = []
blob_list = silver_client.list_blobs(name_starts_with="trans")
for blob in blob_list:
    print(f"Processing {blob.name}")
    blob_client = silver_client.get_blob_client(blob.name)
    blob_data = blob_client.download_blob().readall()
    json_data = json.loads(blob_data.decode("utf-8"))
    
    # Flatten: Extract all roster items, discarding leagueId keys
    for league_id, rosters in json_data.items():
        all_rosters.extend(rosters)  # Add all items to a single list

# Convert to DataFrame and then CSV
if not all_rosters:
    raise ValueError("No roster items found to flatten")
df = pd.DataFrame(all_rosters)
csv_data = df.to_csv(index=False)  # Convert to CSV string without row indices

# Save to silver as CSV 
output_blob_name = "flattened_rosters.csv" 
output_blob_client = silver_client.get_blob_client(output_blob_name)
output_blob_client.upload_blob(csv_data, overwrite=True)
print(f"Uploaded {output_blob_name} to silver container")